In [4]:
import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt

#from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.statespace.sarimax import SARIMAX

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

import pickle
import requests

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

Links
- US vaccine tracker
    - https://www.mayoclinic.org/coronavirus-covid-19/vaccine-tracker/
    - https://usafacts.org/visualizations/covid-vaccine-tracker-states/
- Interactive world wide tracker
    - https://ourworldindata.org/covid-vaccinations
- **vaccine data - https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc
- **cases/deaths - https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36
- **forecast - https://covid.cdc.gov/covid-data-tracker/#forecasting_weeklydeaths
- **hospitalizations - https://gis.cdc.gov/grasp/COVIDNet/COVID19_5.html
- **hopitalizations data - https://www.cdc.gov/coronavirus/2019-ncov/science/forecasting/hospitalizations-forecasts.html
- **the repository - https://github.com/reichlab/covid19-forecast-hub#ensemble-model

http://localhost:8888/notebooks/Documents/UDEMY/UDEMY_TSA_FINAL/UDEMY_TSA_FINAL/08-Prophet-Library/01-Prophet-Forecast-Evaluations.ipynb

https://github.com/nicolasfauchereau/Auckland_Cycling/blob/master/notebooks/paper_day_weather.ipynb

In [3]:
#Read in static data
vax_df = pd.read_csv('data/COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv')
#Keep necessary columns
vax_df = vax_df[['Date', 'Location', 'Dist_Per_100K', 'Admin_Per_100K',
                 'Administered_Dose1_Pop_Pct', 'Series_Complete_Pop_Pct']]

#-- Lowercase all columns to match api format --#

#Create dict for rename
uppercase_columns = vax_df.columns.tolist()
lowercase_columns = [x.lower() for x in vax_df.columns.tolist()]
column_dict = {x: y for x, y in list(zip(uppercase_columns, lowercase_columns))}

#Rename columns and change date format
vax_df.rename(columns = column_dict, inplace = True)
vax_df['date'] = pd.to_datetime(vax_df['date'])
vax_df_clean = vax_df.copy()

vax_df_clean.drop(columns = ['dist_per_100k', 'admin_per_100k'], inplace = True)

#Pickle dataframe
pickle_out = open("pickle/vax_df_clean.pickle","wb")
pickle.dump(vax_df_clean, pickle_out)
pickle_out.close()

In [ ]:
anomaly_revision_hospitalizations = pd.read_csv('https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-anomalies/revisions-inc-hosp.csv')
anomaly_revision_hospitalizations

hospitalizations_df[(hospitalizations_df['location_name'] == 'New Jersey') &
                    (hospitalizations_df['date'] == '2020-11-20')]

#Anomaly revision check
    #Fixes already implemented in final dataframe
anomaly_revision_hospitalizations[(anomaly_revision_hospitalizations['location_name'] == 'New Jersey') &
                                  (anomaly_revision_hospitalizations['date'] == '2020-11-20')]

In [253]:
#pickle_out = open("pickle/vax_df_clean.pickle","wb")
#pickle.dump(vax_df_clean, pickle_out)
#pickle_out.close()

infile = open("pickle/vax_df_clean.pickle",'rb')
vax_df_clean = pickle.load(infile)
infile.close()

vax_df_clean

,date,location,administered_dose1_pop_pct,series_complete_pop_pct
0,2021-07-19,DC,63.0,54.1
1,2021-07-19,MD,63.5,57.9
2,2021-07-19,RP,80.8,74.2
3,2021-07-19,OK,46.3,39.6
4,2021-07-19,WY,40.7,36.0
...,...,...,...,...
14133,2020-12-13,MP,0.0,0.0
14134,2020-12-13,LTC,0.0,0.0
14135,2020-12-13,US,0.0,0.0
14136,2020-12-13,GU,0.0,0.0


In [5]:
def vax_data_update():
    
    '''
    Retrieve updated vax data
    Concat new updates to original dataframe
    Save new dataframe
    '''
    
    #Retrieve api data
    api_url = 'https://data.cdc.gov/resource/unsk-b7fc.json'
    results = requests.get(api_url).json()
    
    #Open pickled data
    infile = open("pickle/vax_df_clean.pickle",'rb')
    vax_df_clean = pickle.load(infile)
    infile.close()
    
    #Append new data to list
    parsed_data = []
    for result in results:
        
        #Retrieve last date on current vax dataset
        last_date = str(vax_df_clean.sort_values('date', ascending = False).iloc[0]['date'])[:10]
        
        #Save data if date new, stop when false
        if result['date'][:10] > last_date:
            data = (result['date'][:10], result['location'],
                    result['administered_dose1_pop_pct'],
                    result['series_complete_pop_pct'])
            parsed_data.append(data)
        else:
            break
    
    #Convert list to dataframe
    new_data = pd.DataFrame(parsed_data,
                 columns = ['date', 'location',
                            'administered_dose1_pop_pct',
                            'series_complete_pop_pct'
                           ]
                )
    
    #Change dtypes
    new_data['date'] = pd.to_datetime(new_data['date'])
    new_data = new_data.astype({'administered_dose1_pop_pct': 'float',
                                'series_complete_pop_pct': 'float'})
    
    #Concat to old dataframe
    vax_df_clean = pd.concat([new_data, vax_df_clean])
    
    #Save results
    pickle_out = open("pickle/vax_df_clean.pickle","wb")
    pickle.dump(vax_df_clean, pickle_out)
    pickle_out.close()
    
    #Return new dataframe
    return vax_df_clean

In [6]:
#States full name. location_name
hospitalizations_df = pd.read_csv('https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-truth/truth-Incident%20Hospitalizations.csv')
#Same as hospitalizations_df
deaths_df = pd.read_csv('https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-truth/truth-Incident%20Deaths.csv')

#States Abbreviated. location_abbreviation
hospitalizations_outliers = pd.read_csv('https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-anomalies/outliers-inc-hosps.csv',
                                        index_col = 0)

broken_dates = hospitalizations_outliers[hospitalizations_outliers['date'] > '2022-01-01'].index
hospitalizations_outliers.drop(index = broken_dates, inplace = True)

#Same as hospitalization_outliers
death_outliers = pd.read_csv('https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-anomalies/outliers-inc-deaths.csv',
                             index_col = 0)

#States are abbreviated
vax_df_clean = vax_data_update()

In [7]:
def table_cleaner(state_abbreviated,
                  hospitalizations_df = hospitalizations_df,
                  deaths_df = deaths_df,
                  hospitalizations_outliers = hospitalizations_outliers,
                  death_outliers = death_outliers,
                  vax_df_clean = vax_df_clean):
    '''
    Clean and concat desired data into Prophet's format
    Returns deaths/hospitalizations w/vax data
    Returns outliers in Prophet's holiday format
    '''
    
    #Import pickled dictionary of state name/abbreviation
    load_states_dict = open("pickle/states.pickle",'rb')
    states = pickle.load(load_states_dict)
    load_states_dict.close()
    
    #Set state name per abbreviation
    full_state = states[state_abbreviated]
    
    #----- Hopitalizations/Fatalities Data -----#
    
    #Retrieve state's hospitalization data
    hosp_df = hospitalizations_df[hospitalizations_df['location_name'] == full_state].sort_values('date')[['date', 'value']]
    #Reformat column names for Prophet
    hosp_df.rename(columns = {'value': 'hospitalizations',
                              'date': 'ds'},
                   inplace = True
                  )
    #Adjust to datetime & set index to date
    hosp_df['ds'] = pd.to_datetime(hosp_df['ds'])
    hosp_df.set_index(keys = 'ds', drop = True, inplace = True)
    
    #Retrieve state's fatality data
    death_df = deaths_df[deaths_df['location_name'] == full_state].sort_values('date')[['date', 'value']]
    #Reformat column names for Prophet
    death_df.rename(columns = {'value': 'deaths',
                               'date': 'ds'},
                   inplace = True
                  )
    #Adjust to datetime & set index to date
    death_df['ds'] = pd.to_datetime(death_df['ds'])
    death_df.set_index(keys = 'ds', drop = True, inplace = True)
    
    #Concat hospitalization/fatality dataframes
    death_hosp = pd.concat([hosp_df, death_df], join = 'outer', axis = 1)
    #Fill early hospital data with 0
    death_hosp['hospitalizations'] = death_hosp['hospitalizations'].fillna(0)
    #Resample as weekly data
    death_hosp = death_hosp.resample('W').mean()
    
    #----- Vax Data -----#
    
    #Retrieve state's vax data
    state_vax = vax_df_clean[vax_df_clean['location'] == state_abbreviated]
    #Reformat column names for Prophet 
    state_vax.rename(columns = {'date': 'ds'},
                    inplace = True
                   )
    #Adjust to datetime & set index to date
    state_vax['ds'] = pd.to_datetime(state_vax['ds'])
    state_vax.set_index(keys = 'ds', drop = True, inplace = True)
    #Resample as weekly data
    state_vax = state_vax.resample('W').mean()
    
    #Concat hospitalization/fatality and vax dataframes
    final_df = pd.concat([death_hosp, state_vax], join = 'outer', axis = 1)
    
    #Fill missing early data with 0's
    columns = ['administered_dose1_pop_pct', 'series_complete_pop_pct']
    final_df[columns] = final_df[columns].fillna(0)
    
    #Drop data missing due to update imbalance
    final_df.dropna(inplace = True)
    final_df.reset_index(inplace = True)
    
    #----- Outlier Data -----#
    
    #Retrieve state's hospitalization outliers
    hosp_out = list(set(hospitalizations_outliers[hospitalizations_outliers['location_abbreviation'] == state_abbreviated]['date']))
    #Format for Prophet
    hosp_out_df = pd.DataFrame({
        'holiday': 'hosp_outliers',
        'ds': pd.to_datetime(hosp_out)    
    })
    #Retrive state's fatality outliers
    deaths_out = list(set(death_outliers[death_outliers['location_abbreviation'] == state_abbreviated]['date']))
    #Format for Prophet
    deaths_out_df = pd.DataFrame({
        'holiday': 'deaths_outliers',
        'ds': pd.to_datetime(hosp_out)    
    })
    #Concat holidays
    outliers = pd.concat([hosp_out_df, deaths_out_df])
    
    
    #return final_df, outliers
    
    pickle_final_df = open(f"state_data_pickled/{state_abbreviated}_final_df.pickle","wb")
    pickle.dump(final_df, pickle_final_df)
    pickle_final_df.close()

    pickle_holidays = open(f"state_data_pickled/{state_abbreviated}_outliers.pickle","wb")
    pickle.dump(outliers, pickle_holidays)
    pickle_holidays.close()

In [8]:
load_states_dict = open("pickle/states.pickle",'rb')
states = pickle.load(load_states_dict)
load_states_dict.close()

for state in list(states.keys()):
    table_cleaner(state)

In [94]:
def make_model(state_abbreviation):
    
    '''
    Run predictions for vax rates
    Use vax predictions for hospitalization/fatality models
    '''
    
    #----- Initial Data Load -----#
    
    load_state_df = open(f'state_data_pickled/{state_abbreviation}_final_df.pickle','rb')
    the_state_df = pickle.load(load_state_df)
    load_state_df.close()

    load_state_outliers = open(f'state_data_pickled/{state_abbreviation}_outliers.pickle','rb')
    state_outliers = pickle.load(load_state_outliers)
    load_state_outliers.close()
    
    #Separate Regressors to run predictions on
    #Rename to Prophet naming conventions
    dose_one_df = the_state_df[['ds', 'administered_dose1_pop_pct']].rename(columns = {'administered_dose1_pop_pct': 'y'})
    series_complete_df = the_state_df[['ds', 'series_complete_pop_pct']].rename(columns = {'series_complete_pop_pct': 'y'})
    
    #----- Regressor Predictions -----#
    
    #Instantiate model for dose one vax
    dose_one_model = Prophet(n_changepoints = 30,
                        seasonality_mode = 'multiplicative',
                        changepoint_prior_scale = 1)

    dose_one_model.fit(dose_one_df[dose_one_df['y'] > 0])
    
    #Make future dataframe and create predictions 1 month forward
    dose_one_future = dose_one_model.make_future_dataframe(periods = 4, freq = 'W')
    dose_one_forecast = dose_one_model.predict(dose_one_future)

    #Instantiate model for complete vax
    series_complete_model = Prophet(n_changepoints = 30,
                        seasonality_mode = 'multiplicative',
                        changepoint_prior_scale = .1)

    series_complete_model.fit(series_complete_df[series_complete_df['y'] > 0])
    
    #Make future dataframe and create predictions 1 month forward
    series_complete_future = series_complete_model.make_future_dataframe(periods = 4, freq = 'W')
    series_complete_forecast = series_complete_model.predict(series_complete_future)

    #Concat predictions
    vax_forecast = pd.concat(
        [dose_one_forecast[['ds', 'yhat']][-4:].\
         set_index(keys = 'ds', drop = True).\
         rename(columns = {'yhat': 'administered_dose1_pop_pct'}),

         series_complete_forecast[['ds', 'yhat']][-4:].\
         set_index(keys = 'ds', drop = True).\
         rename(columns = {'yhat': 'series_complete_pop_pct'})],
        axis = 1
    ).reset_index()

    #Set predictions index to follow full dataframe
    start_index = the_state_df.tail(1).index.values[0] + 1
    vax_forecast.index = pd.RangeIndex(start_index, start_index + 4)
    
    #Concat predictions to end of full dataframe
    the_state_df = pd.concat([the_state_df, vax_forecast])
    
    #Export update dataframe
    pickled_new_state_df = open(f"state_vax_pred_pickled/{state_abbreviation}_vax_pred_df.pickle","wb")
    pickle.dump(the_state_df, pickled_new_state_df)
    pickled_new_state_df.close()
    
    #----- Hospitalization/Fatality Models -----#
    
    #Separate targets
    #Rename to Prophet naming conventions
    state_deaths = the_state_df.drop(columns = 'hospitalizations').rename(columns = {'deaths': 'y'})
    state_hospitalizations = the_state_df.drop(columns = 'deaths').rename(columns = {'hospitalizations': 'y'})
    
    #Covid Fatalities Model
    deaths_model = Prophet(n_changepoints = 30,
                           seasonality_mode = 'multiplicative',
                           changepoint_range = .80,
                           changepoint_prior_scale = 3,
                           holidays = state_outliers[state_outliers['holiday'] == 'deaths_outliers'])
    
    #Add regressors w/ their vax predictions
    deaths_model.add_regressor('administered_dose1_pop_pct')
    deaths_model.add_regressor('series_complete_pop_pct')
    
    #Fit model on current data
    deaths_model.fit(state_deaths[:-4])
    
    #Covid Hospitalizations Model
    hosp_model = Prophet(n_changepoints = 30,
                     seasonality_mode = 'multiplicative',
                     changepoint_range = .98,
                     changepoint_prior_scale = 3,
                     holidays = state_outliers[state_outliers['holiday'] == 'hosp_outliers'])
    
    #Add regressors w/ their vax predictions
    hosp_model.add_regressor('administered_dose1_pop_pct')
    hosp_model.add_regressor('series_complete_pop_pct')
    
    #Fit model on current data
    hosp_model.fit(state_hospitalizations[:-4])
    
    #Export models
    pickle_deaths_model = open(f"models_pickled/{state_abbreviation}_deaths_model.pickle","wb")
    pickle.dump(deaths_model, pickle_deaths_model)
    pickle_deaths_model.close()

    pickle_hosp_model = open(f"models_pickled/{state_abbreviation}_hosp_model.pickle","wb")
    pickle.dump(hosp_model, pickle_hosp_model)
    pickle_hosp_model.close()

In [95]:
load_states_dict = open("pickle/states.pickle",'rb')
states = pickle.load(load_states_dict)
load_states_dict.close()

for state in list(states.keys()):
    make_model(state)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 16.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonali

In [162]:
def visualization_predictions(state_abbreviation, multiplier):
    
    '''
    Create predictions based on vax multiplier
    '''
    
    #----- Data Load In ----- #
    load_deaths_model = open(f'models_pickled/{state_abbreviation}_deaths_model.pickle','rb')
    deaths_model = pickle.load(load_deaths_model)
    load_deaths_model.close()
    
    load_hosp_model = open(f'models_pickled/{state_abbreviation}_hosp_model.pickle','rb')
    hosp_model = pickle.load(load_hosp_model)
    load_hosp_model.close()
    
    load_vax_pred = open(f'state_vax_pred_pickled/{state_abbreviation}_vax_pred_df.pickle','rb')
    vax_pred_df = pickle.load(load_vax_pred)
    load_vax_pred.close()
    
    #Separate deaths/hospitalizations and rename columns
    state_deaths = vax_pred_df.drop(columns = 'hospitalizations').rename(columns = {'deaths': 'y'})
    state_hospitalizations = the_state_df.drop(columns = 'deaths').rename(columns = {'hospitalizations': 'y'})
    
    #----- Fatality Predictions -----#
    
    #Create future dataframe for predictions
    deaths_future = deaths_model.make_future_dataframe(periods = 4, freq = 'W')
    
    #Add regressor columns to future dataframe
    columns = ['administered_dose1_pop_pct', 'series_complete_pop_pct']
    #Multiplier to allow value alterations
    deaths_future[columns] = state_deaths[columns] * multiplier
    #Cap max vaccination rate at 100%
    deaths_future[columns[0]].clip(upper = 100., inplace = True)
    deaths_future[columns[1]].clip(upper = 100., inplace = True)
    
    #Create predictions, cap lower limit as 0
    deaths_forecast = deaths_model.predict(deaths_future)
    deaths_forecast['yhat'].clip(lower = 0, inplace = True)
   
    #----- Hospitalizations Predictions -----#

    #Create future dataframe for predictions
    hosp_future = hosp_model.make_future_dataframe(periods = 4, freq = 'W')
    
    #Add regressor columns to future dataframe
    #Multiplier to allow value alterations
    hosp_future[columns] = state_deaths[columns] * multiplier
    #Cap max vaccination rate at 100%
    hosp_future[columns[0]].clip(upper = 100., inplace = True)
    hosp_future[columns[1]].clip(upper = 100., inplace = True)
    
    #Create predictions, cap lower limit as 0
    hosp_forecast = hosp_model.predict(hosp_future) 
    hosp_forecast['yhat'].clip(lower = 0, inplace = True)        
    
    return deaths_forecast[['ds', 'yhat']], hosp_forecast[['ds', 'yhat']], deaths_future

In [234]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [247]:
state_abbreviation = 'NJ'

# Create figure
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes = True,
                    specs=[[{'type': 'scatter'}],
                           [{'type': 'scatter'}]
                          ],
                    row_heights = [10,
                                   20],
                    subplot_titles=('Vaccination Rate',
                                    'Weekly Hospitalizations & Fatalities')
                   )


# Add traces, one for each slider step
for step in np.arange(0, 2, 0.05):
    
    death_forecast, hosp_forecast, vax_df = visualization_predictions('NJ', step)
    
    fig.add_traces(
        [
        go.Scatter(
            visible = False,
            line = dict(color = '#7dfffc', width=6),
            name = 'Dose 1 Pop. %',
            x = vax_df['ds'],
            y = vax_df['administered_dose1_pop_pct']
        ),
        go.Scatter(
            visible = False,
            line = dict(color = '#7dff80', width=6),
            name = 'Series Complete Pop. %',
            x = vax_df['ds'],
            y = vax_df['series_complete_pop_pct']
        )
        ],
        rows = [1, 1], cols = [1, 1]
    )
    
    fig.add_traces(
        [
        go.Scatter(
            visible = False,
            line = dict(color = '#ff2828', width=6),
            name = 'Fatalities',
            x = death_forecast['ds'],
            y = death_forecast['yhat']
        ),
        go.Scatter(
            visible = False,
            line = dict(color = '#ff9428', width=6),
            name = 'Hospitalizations',
            x = hosp_forecast['ds'],
            y = hosp_forecast['yhat']
        )
        ],
        rows = [2, 2], cols = [1, 1]
    )

# Make middle trace visible
middle_trace = int(len(fig.data)/2)
for i in range(middle_trace, middle_trace + 4):
    fig.data[i].visible = True

# Create and add slider
steps = []
for i in range(0, len(fig.data), 8):
    step = dict(
        method = 'update',
        args = [{'visible': [False] * len(fig.data)},
                {'title': 'Vaccine Rate Multipler at: ' + str(i/80)}],
    )
    step['args'][0]['visible'][i] = True
    
    try:
        step['args'][0]['visible'][i+3] = True
    except:
        continue
    try:
        step['args'][0]['visible'][i+2] = True
    except:
        continue
    try:
        step['args'][0]['visible'][i+1] = True
    except:
        continue
    steps.append(step)

sliders = [dict(
    active = 10,
    currentvalue = {"prefix": "Vaccination Rate Multiplier: "},
    pad = {'t': 50},
    steps = steps
)]

fig.update_layout(
    sliders = sliders,
    yaxis_range = [0, 100]
)

fig.show()